In [7]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl
import aneris

In [8]:
image_path = '../../../data/SOD/model_results/raw/Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx'
mimosa_path = '../../../data/SOD/model_results/raw/MIMOSA_processed_2.xlsx'
gem_e3_path = '../../../data/SOD/model_results/raw/GEM_E3_v2.xlsx'

In [9]:
regions_un = [
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World', 'Africa (UN-R5)',
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)']

regions_chpt_19 = ['IMAGE 3.3|Canada',
            'IMAGE 3.3|Japan',
            'IMAGE 3.3|Mexico',
            'IMAGE 3.3|South Africa',
            'IMAGE 3.3|Turkey',
            'IMAGE 3.3|India',
            'IMAGE 3.3|Rest of Southern Africa',
            'IMAGE 3.3|Brazil',
            'IMAGE 3.3|China Region',
            'IMAGE 3.3|Eastern Africa',
            'IMAGE 3.3|Indonesia Region',
            'IMAGE 3.3|Northern Africa',
            'IMAGE 3.3|Rest of South Asia',
            'IMAGE 3.3|Russia Region',
            'IMAGE 3.3|USA',
            'IMAGE 3.3|Western Africa',
            'IMAGE 3.3|Western Europe',
            'IMAGE 3.3|Oceania']

df_image_all = pyam.IamDataFrame(image_path)

gdp_image = df_image_all.filter(variable=['GDP*'], year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_un = gdp_image.filter(region=regions_un, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image_19 = gdp_image.filter(region=regions_chpt_19, year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
# gdp_image.variable

[INFO] 10:45:56 - pyam.core: Reading file ..\..\..\data\SOD\model_results\raw\Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx


In [10]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [11]:
def compute_3_groups_image(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'CAN',
                                'JAP',
                                'USA',
                                'WEU',
                                'OCE'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'INDO',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'INDIA',
                                'RSAF',
                                'EAF',
                                'NAF',
                                'RSAS',
                                'WAF',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_image(df, variables, weight=None):
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'NAF',
                                    'WAF',
                                    'EAF',
                                    'SAF',
                                    'RSAF',
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'CHN',
                                'INDIA',
                                'INDO',
                                'JAP',
                                'KOR',
                                'ME',
                                'RSAS',
                                'SEAS',
                                'STAN',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                    'CEU',
                                    'RUS',
                                    'UKR',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                    'BRA',
                                    'MEX',
                                    'RCAM',
                                    'RSAM',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                        'CAN',
                                        'OCE',
                                        'TUR',
                                        'USA',
                                        'WEU',
                              ],
                              weight=weight
                              )
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups

def compute_3_groups_gem(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'USA',
                                'CAN',
                                'JPN',
                                'OCE',
                                'EU28'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'MEX',
                                'SAF',
                                'TUR',
                                'BRA',
                                'CHN',
                                'IDN',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'IND',
                                'ROW',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups

def compute_UN_groups_gem(df, variables, weight=None):
    
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'SAF', 
                                    'ROW'
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'JPN', 
                                'CHN',
                                'IND',
                                'KOR',
                                'IDN',
                                'SAR',
                                'REP',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                'REU',
                                'RUS',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                'ARG',
                                'BRA',
                                'MEX',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                'EU28',
                                'USA',
                                'CAN',
                                'TUR',
                                'OCE',
                              ],
                              weight=weight
                              )
    
    df_world = df.filter(variable=variables, region='World')

    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we, df_world])
    return df_un_groups


# Calculate Convertion Ratio

In [12]:

gdp_image_un = compute_UN_groups_image(gdp_image, gdp_image.variable)
df_un_ratio = gdp_image_un.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_un_ppp_input = gdp_image_un.multiply('GDP|PPP', 1, 'GDP|PPP|Input', ignore_units='billion USD_2010/yr')
df_un_image = pyam.concat([df_un_ppp_input, df_un_ratio])

df_3_groups_image = compute_3_groups_image(gdp_image, gdp_image.variable)
df_3_ratio = df_3_groups_image.divide('GDP|PPP', 'GDP|MER', 'GDP|Ratio', ignore_units='')
df_3_ppp_input = df_3_groups_image.multiply('GDP|PPP', 1, 'GDP|PPP|Input', ignore_units='billion USD_2010/yr')
df_3_image = pyam.concat([df_3_ppp_input, df_3_ratio])
df_3_image.region

['High-Income', 'Low-Income', 'Middle-Income']

# MIMOSA and GEM-E3 Income groups

In [14]:
df_unclean_mimosa = pd.read_excel('../../../data/SOD/model_results/raw/MIMOSA_processed_2.xlsx')
ratio_mask = df_unclean_mimosa['Unit'] == 'fraction of GDP'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
# df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]] = df_unclean_mimosa.loc[ratio_mask, df_unclean_mimosa.columns[5:]]  * 100
# df_unclean_mimosa.loc[ratio_mask, 'Unit'] = "% of GDP"

df_mid_clean_mimosa = pyam.IamDataFrame(df_unclean_mimosa)
df_mid_clean_mimosa.rename(scenario={
        "REF": "REF-v2",
        "LIFE-TP": "LIFE-TP-v2",
        "TECH-TP": "TECH-TP-v2",
    }, inplace=True)


df_un_mimosa = compute_UN_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa = compute_3_groups_image(df_mid_clean_mimosa, 'GDP|Climate Damage Corrected|Loss|Relative', weight='GDP|Climate Damage Corrected')
df_3_goups_mimosa.timeseries()

2020  \
model  scenario   region        variable                                   unit                    
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP   0.0   

                                                                                                2030  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003938   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015010   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006037   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.004683   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.018208   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006948   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.003921   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.014938   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.006016   

                                                                                                2040  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007391   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026827   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011207   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011475   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.043599   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.015606   
       TECH-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.007393   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026844   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.011210   

                                                                                                2050  \
model  scenario   region        variable                                   unit                        
MIMOSA LIFE-TP-v2 High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.008042   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.026514   
                  Middle-Income GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.013275   
       REF-v2     High-Income   GDP|Climate Damage Corrected|Loss|Relative fraction of GDP  0.020172   
                  Low-Income    GDP|Climate Damage Corrected|Loss|Relative fr

In [16]:
df_unclean_gem = pd.read_excel('../../../data/SOD/model_results/raw/GEM_E3_v2.xlsx')
df_unclean_gem['Scenario'] = df_unclean_gem['Scenario'].replace({'DECARB_LIFE': 'LIFE-TP-v2',
                                                        'DECARB_TECH': 'TECH-TP-v2',
                                                        'REF_V2':'REF-v2',
                                                  })

df_mid_clean_gem = pyam.IamDataFrame(df_unclean_gem)
df_mid_clean_gem.filter(variable=["GDP|MER", 'Population'])
df_mid_clean_gem.rename(unit={'billion US$2010/yr':'billion USD_2010/yr'}, inplace=True)

df_un_gem = compute_UN_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem = compute_3_groups_gem(df_mid_clean_gem, ["GDP|MER", 'Population'])
df_3_groups_gem.timeseries()

2015  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  41527.174647   
                                      Population million               1032.375424   
                        Low-Income    GDP|MER    billion USD_2010/yr   8947.068280   
                                      Population million               3409.710058   
                        Middle-Income GDP|MER    billion USD_2010/yr  17914.158252   
                                      Population million               2270.461976   

                                                                              2020  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582509   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152858   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710837   
                                      Population million               2343.921774   
             REF-v2     High-Income   GDP|MER    billion USD_2010/yr  42001.582508   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152884   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710853   
                                      Population million               2343.921774   
             TECH-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  42001.582510   
                                      Population million               1051.723709   
                        Low-Income    GDP|MER    billion USD_2010/yr  10261.152940   
                                      Population million               3684.994699   
                        Middle-Income GDP|MER    billion USD_2010/yr  21198.710855   
                                      Population million               2343.921774   

                                                                              2025  \
model        scenario   region        variable   unit                                
GEM-E3_V2023 LIFE-TP-v2 High-Income   GDP|MER    billion USD_2010/yr  47311.769790   
                                      Population million               1066.676412   
                        Low-Income    GDP|MER    billion USD_2010/yr  13270.101082   
                                      Population million               3955.578312   
                        Middle-Income GDP|MER    billion USD_2010/yr  26908.478088   
                                  

# GEM-E3 MER to PPP

In [17]:
df_un_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_un_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_un_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_un = pyam.concat([df_un_gem, df_un_mimosa, df_un_image])
df_un = df_un.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_un_to_harmonize = df_un.multiply('GDP|MER', 'GDP|Ratio', 'GDP|PPP|Narrative Effects', ignore_units='billion USD_2010/yr')

df_un_hist = df_un.filter(variable='GDP|PPP|Input')
df_un_hist.rename(variable={'GDP|PPP|Input':'GDP|PPP|Narrative Effects'}, inplace=True)
df_un_to_harmonize.filter(region='Africa*').timeseries()

2020  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572373   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572436   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  17561.572573   

                                                                                               2030  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25612.682806   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25943.590676   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  25602.083311   

                                                                                               2040  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  36745.165973   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  37548.215430   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  36718.849664   

                                                                                               2050  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  50895.211779   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  51843.008527   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  51018.948032   

                                                                                               2060  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  67631.790112   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  68442.299022   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  67889.959674   

                                                                                               2070  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  77576.519724   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  78074.607651   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  78064.152736   

                                                                                               2080  \
model         scenario   region         variable                  unit                                
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  88017.698355   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  87555.626546   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  88653.407175   

                                                                                                2090  \
model         scenario   region         variable                  unit                                 
Macro_Economy LIFE-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr  100069.881717   
              REF-v2     Africa (UN-R5) GDP|PPP|Narrative Effects billion USD_2010/yr   98158.467924   
              TECH-TP-v2 Africa (UN-R5) GDP|PPP|Narrative Effects b

In [18]:
df_3_groups_gem.rename(model={'GEM-E3_V2023':'Macro_Economy'}, inplace=True)
df_3_goups_mimosa.rename(model={'MIMOSA':'Macro_Economy'}, inplace=True)
df_3_image.rename(model={'IMAGE 3.3':'Macro_Economy'}, inplace=True)

df_3 = pyam.concat([df_3_groups_gem, df_3_goups_mimosa, df_3_image])
df_3 = df_3.filter(year=[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])
df_3_to_harmonize = df_3.multiply('GDP|MER', 'GDP|Ratio', 'GDP|PPP|Narrative Effects', ignore_units='billion USD_2010/yr')
df_3_hist = df_3.filter(variable='GDP|PPP|Input')
df_3_hist.rename(variable={'GDP|PPP|Input':'GDP|PPP|Narrative Effects'}, inplace=True)
df_3_to_harmonize.timeseries()

2020  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526929   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380403   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294892   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526928   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380475   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294926   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  39300.526930   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  28167.380629   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  45557.294931   

                                                                                              2030  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48037.407112   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  42753.038969   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64205.077193   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48356.838236   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  43418.455230   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64873.264391   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  48050.864091   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  42781.188188   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  64252.098987   

                                                                                              2040  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  56544.660493   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  61436.486618   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  74947.153277   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  57208.903947   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  63077.823430   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  77152.522680   
              TECH-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  56608.457286   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  61476.534061   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  75174.998173   

                                                                                              2050  \
model         scenario   region        variable                  unit                                
Macro_Economy LIFE-TP-v2 High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  66870.933605   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  85019.349030   
                         Middle-Income GDP|PPP|Narrative Effects billion USD_2010/yr  84436.730729   
              REF-v2     High-Income   GDP|PPP|Narrative Effects billion USD_2010/yr  67858.656195   
                         Low-Income    GDP|PPP|Narrative Effects billion USD_2010/yr  87118.821336   
           

# Harmonize to SSP2 Input

In [19]:
# AIM will be harmonized to IMAGE REF historic values
def harmonize_df(df_gem, df_image_hist):

    # # Create a dataframe that contains the method of harmonization per variable
    # # Only create methods for variables present in AIM
    methods = df_gem.timeseries().reset_index(level='model', drop=True)
    # The offset methods ensure internal regional consistency better than the ratio methods
    methods['method'] = 'constant_offset' # 'reduce_ratio_2080' 'constant_ratio' 'constant_offset' 'reduce_offset_2080'
    methods_series = methods['method']

    hist_to_harmonize = df_image_hist.timeseries()
    df_gem_to_harmonize = df_gem.timeseries()

    # Remove the model level from index and column names should be strings
    df_gem_to_harmonize = df_gem_to_harmonize.reset_index(level='model', drop=True)
    df_gem_to_harmonize.columns = df_gem_to_harmonize.columns.astype(str)
    hist_to_harmonize = hist_to_harmonize.reset_index(level='model', drop=True)
    hist_to_harmonize.columns = hist_to_harmonize.columns.astype(str)

    # Harmonize 
    h = aneris.harmonize.Harmonizer(df_gem_to_harmonize, hist_to_harmonize, config={'harmonize_year': '2020'})
    harmonized = h.harmonize(methods_series) # methods_copy

    # Add all necessary index columns
    new_level = 'Macro_Economy'
    df_index = harmonized.index.to_frame()  # Convert the MultiIndex to a DataFrame
    df_index['model'] = new_level  # Add the new level
    df_index = df_index[['model', 'scenario', 'region', 'variable', 'unit']]
    new_index = pd.MultiIndex.from_frame(df_index)  # Create a new MultiIndex with the added level
    harmonized.index = new_index

    df_harmonized = pyam.IamDataFrame(harmonized)
    return df_harmonized

df_un_ppp_harmonized = harmonize_df(df_un_to_harmonize, df_un_hist)
df_3_ppp_harmonized = harmonize_df(df_3_to_harmonize, df_3_hist)

INFO:root:Harmonizing with constant_offset
INFO:root:Harmonizing with constant_offset


In [20]:
df_un_harmonized = pyam.concat([df_un, df_un_ppp_harmonized])

df_un_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_un_harmonized.multiply('GDP|PPP|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|PPP|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)
df_un_harmonized.filter(region='World*').timeseries()

2020  \
model         scenario   region variable                                           unit                                 
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288125   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   
              REF-v2     World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288172   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   
              TECH-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.000000   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           1.000000   
                                GDP|MER                                            billion USD_2010/yr   80110.288231   
                                GDP|PPP|Input                                      billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  102473.678217   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  102473.678217   
                                GDP|Ratio                                                                    1.460084   
                                Population                                         million                7795.136232   

                                                                                                                 2030  \
model         scenario   region variable                                           unit                                 
Macro_Economy LIFE-TP-v2 World  GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP           0.007138   
                                GDP|Climate Damage Corrected|Loss|Relative (Inv...                           0.992862   
                                GDP|MER                                            billion USD_2010/yr  109010.301617   
                                GDP|PPP|Input                                      billion USD_2010/yr  146035.354670   
                                GDP|PPP|Narrative Effects                          billion USD_2010/yr  148724.513896   
                                GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  147662.919481  

In [21]:
df_3_harmonized = pyam.concat([df_3, df_3_ppp_harmonized])

df_3_harmonized.subtract(1, 
               'GDP|Climate Damage Corrected|Loss|Relative', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)',
               ignore_units='',
               append=True 
              )
df_3_harmonized.multiply('GDP|PPP|Narrative Effects', 
               'GDP|Climate Damage Corrected|Loss|Relative (Inverted)', 
               'GDP|PPP|Narrative and Climate Effects', 
               ignore_units='billion USD_2010/yr', 
               append=True)

df_3_harmonized.filter(region='Low-Income').timeseries()

2020  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152858   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              REF-v2     Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152884   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   
              TECH-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.000000   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          1.000000   
                                    GDP|MER                                            billion USD_2010/yr  10261.152940   
                                    GDP|PPP|Input                                      billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  12910.957393   
                                    GDP|PPP|Narrative and Climate Effects              billion USD_2010/yr  12910.957393   
                                    GDP|Ratio                                                                   2.745050   
                                    Population                                         million               3684.994699   

                                                                                                                    2030  \
model         scenario   region     variable                                           unit                                
Macro_Economy LIFE-TP-v2 Low-Income GDP|Climate Damage Corrected|Loss|Relative         fraction of GDP          0.015010   
                                    GDP|Climate Damage Corrected|Loss|Relative (Inv...                          0.984990   
                                    GDP|MER                                            billion USD_2010/yr  16823.957896   
                                    GDP|PPP|Input                                      billion USD_2010/yr  22732.435364   
                                    GDP|PPP|Narrative Effects                          billion USD_2010/yr  27496.615959   
                       

In [22]:
df_total = pyam.concat([df_un_harmonized, df_3_harmonized])
df_total.rename(scenario={
    'LIFE-TP-v2':'LIFE-TP',
    'REF-v2':'REF',
    'TECH-TP-v2':'TECH-TP'
}, inplace=True)

df_total.divide('GDP|PPP|Input', "Population", 'GDP|PPP|Input|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|PPP|Narrative Effects', "Population", 'GDP|PPP|Narrative Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)
df_total.divide('GDP|PPP|Narrative and Climate Effects', "Population", 'GDP|PPP|Narrative and Climate Effects|Per Capita', ignore_units='thousand USD_2010/yr', append=True)

df_total = df_total.timeseries().reset_index()
df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_10868\3091355377.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_total.loc[:, df_total.columns[5:]] = df_total.loc[:, df_total.columns[5:]].applymap(round_to_0_1_percent)


In [23]:
df_total_un = df_total[df_total['region'].isin([
    'Africa (UN-R5)', 
    'Asia and the Pacific (UN-R5)',
    'Eastern Europe (UN-R5)',
    'Latin America and Caribbean (UN-R5)',
    'Western Europe and Other States (UN-R5)',
    'World'])]
df_total_un = df_total_un[df_total_un['variable'].isin([
    'GDP|PPP|Input',
    # 'GDP|PPP|Narrative Effects',
    'GDP|PPP|Narrative and Climate Effects',
    'GDP|PPP|Input|Per Capita',
    # 'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|PPP|Narrative and Climate Effects|Per Capita'
    ])].reset_index(drop=True)
df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_20.xlsx', index=False)
# df_total_un.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_14.xlsx', index=False)
df_total_un

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Input,billion USD_2010/yr,5570.00,8630.00,14300.00,23300.00,37500.00,58400.0,87400.0,125000.0,171000.0
1,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Input|Per Capita,thousand USD_2010/yr,2.36,3.10,4.42,6.44,9.35,13.4,18.8,25.7,34.0
2,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,5570.00,13400.00,24100.00,37900.00,54500.00,64400.0,74900.0,87000.0,100000.0
3,Macro_Economy,LIFE-TP,Africa (UN-R5),GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,2.36,4.82,7.47,10.50,13.60,14.8,16.2,17.9,19.9
4,Macro_Economy,LIFE-TP,Asia and the Pacific (UN-R5),GDP|PPP|Input,billion USD_2010/yr,48900.00,77000.00,104000.00,130000.00,155000.00,181000.0,206000.0,231000.0,256000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Macro_Economy,TECH-TP,Western Europe and Other States (UN-R5),GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,34.50,41.40,47.70,56.60,67.20,75.3,83.7,92.7,102.0
68,Macro_Economy,TECH-TP,World,GDP|PPP|Input,billion USD_2010/yr,102000.00,146000.00,191000.00,239000.00,292000.00,353000.0,423000.0,501000.0,587000.0
69,Macro_Economy,TECH-TP,World,GDP|PPP|Input|Per Capita,thousand USD_2010/yr,13.10,17.10,20.70,24.50,28.70,33.7,39.4,46.1,53.7
70,Macro_Economy,TECH-TP,World,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,102000.00,148000.00,190000.00,240000.00,299000.00,344000.0,391000.0,441000.0,492000.0


In [24]:
df_total_income = df_total[df_total['region'].isin([
    'High-Income', 
    'Low-Income',
    'Middle-Income',
    ])]
df_total_income = df_total_income[df_total_income['variable'].isin([
    'GDP|PPP|Input',
    # 'GDP|PPP|Narrative Effects',
    'GDP|PPP|Narrative and Climate Effects',
    'GDP|PPP|Input|Per Capita',
    # 'GDP|PPP|Narrative Effects|Per Capita',
    'GDP|PPP|Narrative and Climate Effects|Per Capita'
    ])].reset_index(drop=True)
df_total_income.to_excel('../../../data/SOD/model_results/to_share/GDP_to_share_chapter_19.xlsx', index=False)
df_total_income

,model,scenario,region,variable,unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Input,billion USD_2010/yr,37900.0,45700.00,52800.00,60000.0,67600.0,76000.0,84300.0,92500.0,101000.0
1,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Input|Per Capita,thousand USD_2010/yr,36.0,42.40,47.90,54.2,60.8,68.1,75.1,81.7,88.0
2,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,37900.0,46400.00,54700.00,64900.0,77200.0,86600.0,96700.0,107000.0,119000.0
3,Macro_Economy,LIFE-TP,High-Income,GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,36.0,43.00,49.70,58.6,69.5,77.7,86.1,94.8,104.0
4,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Input,billion USD_2010/yr,12900.0,22700.00,36500.00,55900.0,82600.0,118000.0,162000.0,216000.0,279000.0
5,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Input|Per Capita,thousand USD_2010/yr,3.5,5.38,7.68,10.8,14.8,20.0,26.5,34.3,43.5
6,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Narrative and Climate Effects,billion USD_2010/yr,12900.0,27100.00,44900.00,67900.0,94200.0,113000.0,134000.0,156000.0,179000.0
7,Macro_Economy,LIFE-TP,Low-Income,GDP|PPP|Narrative and Climate Effects|Per Capita,thousand USD_2010/yr,3.5,6.41,9.46,13.1,16.9,19.2,21.9,24.8,28.0
8,Macro_Economy,LIFE-TP,Middle-Income,GDP|PPP|Input,billion USD_2010/yr,36200.0,56000.00,73000.00,86900.0,97300.0,106000.0,113000.0,119000.0,125000.0
9,Macro_Economy,LIFE-TP,Middle-Income,GDP|PPP|Input|Per Capita,thousand USD_2010/yr,15.4,23.10,29.80,35.8,41.2,46.7,52.0,57.1,62.4
